In [179]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from pathlib import Path

df = pd.read_csv(r"/content/drive/MyDrive/Wild Code School/PROJET_2/CSV/df_ML.csv", index_col = 0)

In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 157773 entries, 0 to 157772
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   genres             157773 non-null  object 
 1   Director_Names     156450 non-null  object 
 2   Actor_Names        144845 non-null  object 
 3   année_de_sortie    157773 non-null  int64  
 4   runtime            157773 non-null  int64  
 5   averageRating      157773 non-null  float64
 6   tconst             157773 non-null  object 
 7   title_fr           157773 non-null  object 
 8   overview           157773 non-null  object 
 9   processed          157770 non-null  object 
 10  combined_features  157773 non-null  object 
 11  main_genre         157773 non-null  object 
dtypes: float64(1), int64(2), object(9)
memory usage: 15.6+ MB


In [182]:
df

,genres,Director_Names,Actor_Names,année_de_sortie,runtime,averageRating,tconst,title_fr,overview,processed,combined_features,main_genre
0,[Comedy],['Frank R. Strayer'],"['Penny Singleton', 'Arthur Lake', 'Larry Simm...",1938,70,6.9,tt0029927,Blondie,Blondie and Dagwood are about to celebrate the...,blondie dagwood celebrate fifth wedding annive...,Blondie Comedy Penny Singleton Arthur Lake L...,Comedy
1,"[Drama, Romance]","['Shintarô Ishihara', 'Marcel Ophüls', 'Renzo ...","['Jean-Pierre Léaud', 'Marie-France Pisier', '...",1962,110,7.2,tt0055747,L'amour à vingt ans,Love at Twenty unites five directors from five...,love twenty unites five directors five differe...,Lamour à vingt ans Drama Romance Jean-Pierre ...,Drama
2,"[Drama, Comedy, Crime]",['Aki Kaurismäki'],"['Turo Pajala', 'Susanna Haavisto', 'Matti Pel...",1988,73,7.5,tt0094675,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,taisto kasurinen finnish coal miner whose fath...,Ariel Drama Comedy Crime Turo Pajala Susann...,Drama
3,"[Drama, Comedy, Romance]",['Aki Kaurismäki'],"['Matti Pellonpää', 'Kati Outinen', 'Sakari Ku...",1986,76,7.5,tt0092149,Ombres au paradis,"An episode in the life of Nikander, a garbage ...",episode life nikander garbage man involving de...,Ombres au paradis Drama Comedy Romance Matti...,Drama
4,"[Crime, Comedy]","['Allison Anders', 'Alexandre Rockwell', 'Robe...","['Tim Roth', 'Antonio Banderas', 'Sammi Davis'...",1995,98,6.7,tt0113101,Groom service,It's Ted the Bellhop's first night on the job....,ted bellhop first night job hotel unusual gues...,Groom service Crime Comedy Tim Roth Antonio ...,Crime
...,...,...,...,...,...,...,...,...,...,...,...,...
157768,[Drama],['Patricia Ayala Ruiz'],NaN,2023,90,6.6,tt27219003,Puentes en el mar,"Like every day, Alicia takes her son to school...",like every day alicia takes son school michael...,Puentes en el mar Drama Patricia Ayala Ruiz 2...,Drama
157769,"[Crime, Drama]","['Ka-Wai Kam', 'Jing Wong']","['Bo Yuen Chan', 'Jordan Chan', 'Weichuan Guo'...",2022,71,0.0,tt24962414,Zhong zhuang zhan jing,The film tells the story of the heavy-duty pol...,film tells story police chief inspector chen z...,Zhong zhuang zhan jing Crime Drama Bo Yuen Ch...,Crime
157770,"[Action, War]",['Denis Neymand'],"['Aleksandr Baranovskiy', 'Oleg Chernov', 'Evg...",2021,100,5.4,tt17352384,Granit,Mozambique requests from Russia is being helpe...,mozambique requests russia helped fight milita...,Granit Action War Aleksandr Baranovskiy Oleg...,Action
157771,"[Documentary, History]",['Germán Roda'],"['Pepe Viyuela', 'Laura Gómez-Lacueva', 'Alber...",2020,83,7.2,tt8223104,"Marcelino, el mejor payaso del mundo","An account of the life, work and many triumphs...",account life work many triumphs spanish clown ...,Marcelino el mejor payaso del mundo Documenta...,Documentary


In [162]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['combined_features'])

In [163]:
knn = NearestNeighbors(metric='cosine',n_neighbors=50)
knn.fit(tfidf_matrix)

NearestNeighbors(metric='cosine', n_neighbors=50)

In [164]:
tfidf_matrix.shape

(157773, 382528)

In [185]:
def get_recommendations(title):
    # Trouver l'indice du titre dans le dataframe
    matching_titles = df[df['title_fr'].str.lower().str.contains(title.lower())]
    if matching_titles.empty:
        print(f"The title '{title}' was not found in the dataset.")
        return None

    id_title = matching_titles.index[0]
    distances, indices = knn.kneighbors(tfidf_matrix[id_title])

    # Indices des films similaires
    similar_movies_indices = indices.flatten()[1:]

    # Retirer le film lui-même
    similar_movies = df.iloc[similar_movies_indices]

    return similar_movies

In [188]:
title_raw = input("Enter a movie name: ")
title_x = title_raw.lower()
# Exemple d'utilisation
recommended_movies = get_recommendations(title_x)
print(recommended_movies[['title_fr']])

Enter a movie name: Killing Fields
                                title_fr
142903                     Jackie & Ryan
76062                           Kick-Ass
54270                              Greta
73878          Entering the Spirit Realm
124337                       Dark Places
93925                         Kick-Ass 2
124693             The Grace Lee Project
150770                     Brain on Fire
115213                 Home Run Showdown
52613              South Bureau Homicide
78638                          Fall Time
47538                            The 355
147840                   All Summers End
147099                November Criminals
106625                              Hick
25933                        Small Group
2898                           SLC Punk!
91775       Papa, ses embrouilles et moi
54355                  I Love You, Daddy
14847                               Fall
94521   Mon copain Mac héros des étoiles
3584                             The Eye
4317                  